**Table of contents**<a id='toc0_'></a>    
- [Import packages](#toc1_)    
- [Set up](#toc2_)    
  - [Define paths](#toc2_1_)    
- [Load raw footfall data](#toc3_)    
- [Raw data stats](#toc4_)    
  - [summary](#toc4_1_)    
  - [check duplicates](#toc4_2_)    
- [Process duplicated sensors in sensor location data](#toc5_)    
- [Split historical footfall data by year](#toc6_)    
- [Merge historical footfall data, current footfall data and sensor location data](#toc7_)    
  - [status](#toc7_1_)    
  - [double check](#toc7_2_)    
  - [Split data based on covid period](#toc7_3_)    
  - [Split data based on year](#toc7_4_)    
- [Handling missing values (without interpolation)](#toc8_)    
  - [Extract year span for each sensor](#toc8_1_)    
    - [Save data](#toc8_1_1_)    
  - [Calculate missing rate and plot data](#toc8_2_)    
  - [Remove sensor with missing data rate >= 50%](#toc8_3_)    
  - [Create segments](#toc8_4_)    
- [Process features](#toc9_)    
  - [funs](#toc9_1_)    
  - [fetch nearby amenity counts](#toc9_2_)    
  - [fetch nearby amenities for each sensor](#toc9_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [25]:
# !pip install -q tsfresh stumpy mplcursors plotly kaleido # colab
# # !pip install -q tsfresh==0.20.1 stumpy==1.12.0 missingno==0.5.2 geopy==2.4.0 mplcursors==0.5.2 # local machine

# <a id='toc1_'></a>[Import packages](#toc0_)

In [87]:
import warnings
warnings.filterwarnings("ignore")

import folium
import os
import osmnx as ox
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import plotly.express as px
import missingno as msno
import seaborn as sns
from scipy.spatial import distance
from pathlib import Path
from datetime import timedelta, datetime
from geopy.distance import geodesic
from tsfresh import extract_features
from tsfresh.feature_selection.selection import select_features
from tsfresh.utilities.dataframe_functions import impute
from math import radians, cos, sin, asin, sqrt
from tabulate import tabulate
from importlib import reload
from collections import defaultdict

import sys
if Path('/content/drive/MyDrive').exists():
  sys.path.append('/content/drive/MyDrive/Colab Notebooks/custom_modules')
else:
  sys.path.append('./custom_modules')

import basic_funs, plot_funs
reload(basic_funs)
reload(plot_funs)
from basic_funs import *
from plot_funs import *

# <a id='toc2_'></a>[Set up](#toc0_)

In [70]:
save_subdirs = {
  "original": "0. original_data",
  "merged": "1. merged_peds_data_hist_curr",
  "missing_rate": "2. missing_rate",
  "remove_sensors": "3. remove_sensors_with_high_missing_rate",
  "final_group": "4. final_group",
  "add_fea": "5. join_features",
}

save_subdirs = Config(**save_subdirs)

rewrite = True

## <a id='toc2_1_'></a>[Define paths](#toc0_)

In [71]:
footfall_counts_path_min = 'pedestrian-counting-system-past-hour-counts-per-minute'
footfall_counts_path1 = 'pedestrian-counting-system-monthly-counts-per-hour'
footfall_counts_path2 = 'Pedestrian_Counting_System_Monthly_counts_per_hour_may_2009_to_14_dec_2022'
sensor_locations_path = 'pedestrian-counting-system-sensor-locations'

local_path = Path('../Data (20230918)')
drive_path = Path('/content/drive/MyDrive/Data/Melbourne_Footfalls')

base_path = local_path if local_path.exists() else drive_path

In [72]:
save_dir = Path('../data_preprocessed') if local_path.exists() else Path('/content/drive/MyDrive/ProcessedData_Melbourne_Footfalls')

if save_dir.exists() == False:
  save_dir.mkdir(parents=True, exist_ok=True)

# <a id='toc3_'></a>[Load raw footfall data](#toc0_)

[Pedestrian Counting System (counts per hour)](https://melbournetestbed.opendatasoft.com/explore/dataset/pedestrian-counting-system-monthly-counts-per-hour/information/)

[Pedestrian Counting System - Sensor Locations](https://melbournetestbed.opendatasoft.com/explore/dataset/pedestrian-counting-system-sensor-locations/information/)

the new data (collected since 2023) has an ISO 8601 datetime format with a timezone offset

while the historical data hasn't timezone-awareness

In [57]:
footfall_counts_23_today = pd.read_excel(base_path.joinpath(footfall_counts_path1 + '.xlsx'))
footfall_counts_min = pd.read_excel(base_path.joinpath(footfall_counts_path_min + '.xlsx'))
# footfall_counts_09_22 = pd.read_csv(base_path.joinpath(footfall_counts_path2 + '.csv'))
footfall_counts_09_22 = read_file_with_stem(base_path, footfall_counts_path2)
sensor_locations = pd.read_excel(base_path.joinpath(sensor_locations_path + '.xlsx'))

find matched file(s): [PosixPath('../Data (20230918)/Pedestrian_Counting_System_Monthly_counts_per_hour_may_2009_to_14_dec_2022.csv'), PosixPath('../Data (20230918)/Pedestrian_Counting_System_Monthly_counts_per_hour_may_2009_to_14_dec_2022.csv.gz')]
read ../Data (20230918)/Pedestrian_Counting_System_Monthly_counts_per_hour_may_2009_to_14_dec_2022.csv


# <a id='toc4_'></a>[Raw data stats](#toc0_)

## <a id='toc4_1_'></a>[summary](#toc0_)

In [ ]:
data_stats = {
  "Data Description": [
    "footfall_counts (2009 - 2022)",
    "footfall_counts (2023 - today)",
    "footfall_counts (past hour per min)",
    "sensor_locations"
  ],
  "Size": [
    footfall_counts_09_22.shape,
    footfall_counts_23_today.shape,
    footfall_counts_min.shape,
    sensor_locations.shape
  ],
  "Attributes": [
    footfall_counts_09_22.columns.tolist(),
    footfall_counts_23_today.columns.tolist(),
    footfall_counts_min.columns.tolist(),
    sensor_locations.columns.tolist()
  ],
  "No_Sensor_ID_or_Location_ID": [
    len(footfall_counts_09_22.Sensor_ID.unique()),
    len(footfall_counts_23_today.LocationID.unique()),
    len(footfall_counts_min.Location_ID.unique()),
    len(sensor_locations.Location_ID.unique()),
  ],
  "No_Sensor_Name": [
    len(footfall_counts_09_22.Sensor_Name.unique()),
    np.nan,
    np.nan,
    len(sensor_locations.Sensor_Name.unique()),
  ],
  "No_Sensor_Description": [
    np.nan,
    np.nan,
    np.nan,
    len(sensor_locations.Sensor_Description.unique()),
  ]
}

df_summary = pd.DataFrame(data_stats)
save_data(df_summary, save_dir, save_subdirs.original, 'data_summary.xlsx', rewrite=rewrite)
df_summary

## <a id='toc4_2_'></a>[check duplicates](#toc0_)

especially for data of sensor location

duplicate location of sensor location data:

- Harbour Esplanade (Location_ID: 111, 120 with same geo location)

- Birrarung Marr (Location_ID: 7, 122 with same geo location)

- Birrarung Marr East - Batman Ave Bridge Entry (Location_ID: 123, 124 with different geo location)


In [ ]:
duplicates_23_today = check_duplicates(footfall_counts_23_today, print_msg='footfall_counts_23_today')
duplicates_min = check_duplicates(footfall_counts_min, print_msg='footfall_counts_min')
duplicates_09_22 = check_duplicates(footfall_counts_09_22, print_msg='footfall_counts_09_22')

# duplicates in sensor location data
columns_to_check = [
  'all',
  ['Sensor_Description'],
  ['Sensor_Name'],
  ['Sensor_Name', 'Location'],
  ['Sensor_Description', 'Location'],
  ['Location'],
  ['Location_ID']
]

duplicate_dfs = [check_duplicates(sensor_locations, col, print_msg='sensor_locations') for col in columns_to_check]

duplicates_sensor_locations = pd.concat(duplicate_dfs, axis=0)
duplicates_sensor_locations.drop_duplicates(inplace=True)
duplicates_sensor_locations.dropna(axis=1, how='all', inplace=True)
save_data(duplicates_sensor_locations, save_dir, save_subdirs.original, 'duplicates_sensor_locations.xlsx', index=True, rewrite=rewrite)

# <a id='toc5_'></a>[Process duplicated sensors in sensor location data](#toc0_)

the current data only has LocationID without sensor description and sensor name

In [ ]:
# duplicates_sensor_locations.reset_index(drop=True, inplace=True)
duplicates_sensor_locations

In [ ]:
# check if these id in historical data and current data
print(footfall_counts_09_22[footfall_counts_09_22['Sensor_ID'].isin(duplicates_sensor_locations['Location_ID'])]['Sensor_ID'].unique())
print(footfall_counts_23_today[footfall_counts_23_today['LocationID'].isin(duplicates_sensor_locations['Location_ID'])]['LocationID'].unique())

In [ ]:
# find sensors with same geo location but different Location_ID / Sensor_Description / Sensor_Name
duplicate_sensors = duplicates_sensor_locations[duplicates_sensor_locations[['Location']].duplicated(keep=False)]
duplicate_sensors

In [ ]:
duplicates_sensor_locations[~duplicates_sensor_locations[['Location']].duplicated(keep=False)]

If the Sensor_Description is the same, but the Sensor_Name and Location_ID are different, the names and IDs should be aggregated together in the New_Sensor_Name column.

For the same Sensor_Description, the Sensor_Name and Location_ID pairs should be ordered by Location_ID.

In [ ]:
processed_sensors = process_sensor_loc_data(sensor_locations)
save_data(processed_sensors, save_dir, save_subdirs.merged, 'sensor_locations_processed.xlsx')

then double check if the process correctly

In [ ]:
processed_sensors[processed_sensors['Sensor_Name'] == 'HarbBlix_T']

In [ ]:
processed_sensors[processed_sensors['Sensor_Description'] == 'Birrarung Marr']

In [ ]:
processed_sensors[processed_sensors['Sensor_Name'] == 'BirBridge_T']

In [ ]:
processed_sensors[processed_sensors['Sensor_Name'] == 'BirBridge_T']['New_Sensor_Name'].unique()

# <a id='toc6_'></a>[Split historical footfall data by year](#toc0_)

In [ ]:
for year, group in footfall_counts_09_22.groupby('Year'):
  save_data(group, save_dir, save_subdirs.original, f"data_for_year_{year}.xlsx", rewrite=rewrite)

footfall_counts_23_today['SensingDateTime(Hour)'] = pd.to_datetime(footfall_counts_23_today['SensingDateTime(Hour)'], utc=True)
footfall_counts_23_today['SensingDateTime(Hour)'] = footfall_counts_23_today['SensingDateTime(Hour)'].dt.tz_convert('Australia/Sydney')
footfall_counts_23_today['SensingDateTime(Hour)'] = footfall_counts_23_today['SensingDateTime(Hour)'].dt.tz_localize(None)

save_data(footfall_counts_23_today, save_dir, save_subdirs.original, "data_for_year_2023.xlsx", rewrite=rewrite)

# <a id='toc7_'></a>[Merge historical footfall data, current footfall data and sensor location data](#toc0_)

footfall_counts_09_22, footfall_counts_23_today and sensor_locations have different column names, so have to think a way to unify the column names for further processing
- footfall_counts_09_22: Sensor_ID, Sensor_Name
- footfall_counts_23_today: LocationID
- sensor_locations: Location_ID, Sensor_Description, Sensor_Name

(the Sensor_Description of sensor_locations seems similar to Sensor_Name of footfall_counts_23_today)

['Sensor_Name', 'Sensor_ID'] in footfall_counts_09_22 is equivalent to ['Sensor_Description', 'Location_ID'] in sensor_locations

In [ ]:
hist_df, curr_df = format_datetime_remove_duplicates(footfall_counts_09_22, footfall_counts_23_today)
save_data(hist_df, save_dir, save_subdirs.merged, 'footfall_counts_09_22.csv', rewrite=rewrite)
save_data(curr_df, save_dir, save_subdirs.merged, 'footfall_counts_23_today.xlsx', rewrite=rewrite)

In [ ]:
# The pair of ['Sensor_Name', 'Sensor_ID'] in hist_df should equivalent to
# ['Sensor_Description', 'Location_ID'] in sensor data.
final_merged_df, uncommon_pairs = merge_hist_curr_sensor_data(hist_df, curr_df, processed_sensors)

In [ ]:
save_data(final_merged_df, save_dir, save_subdirs.merged, "footfall_merged.csv", rewrite=rewrite)

## <a id='toc7_1_'></a>[status](#toc0_)

In [ ]:
uncommon_pairs

In [ ]:
# find the rows where the values in the LocationID column of footfall_counts_23_today are NOT found
# in the Location_ID column of the sensor_locations DataFrame.
footfall_counts_23_today[~footfall_counts_23_today['LocationID'].isin(sensor_locations['Location_ID'])]

## <a id='toc7_2_'></a>[double check](#toc0_)

In [ ]:
unique_sensor_ids1 = footfall_counts_09_22[~footfall_counts_09_22['Sensor_ID'].isin(sensor_locations['Location_ID'])]['Sensor_ID'].unique()
unique_sensor_ids2 = footfall_counts_23_today[~footfall_counts_23_today['LocationID'].isin(sensor_locations['Location_ID'])]['LocationID'].unique()
unique_sensor_ids = list(unique_sensor_ids1) + list(unique_sensor_ids2)
removed = footfall_counts_09_22['Sensor_ID'].isin(unique_sensor_ids).sum() + footfall_counts_23_today['LocationID'].isin(unique_sensor_ids).sum()
print(f"These sensors may have been removed: {unique_sensor_ids}")
print(f"{removed} data may be removed.")

In [ ]:
final_merged_df.shape

In [ ]:
footfall_counts_09_22.shape[0] + footfall_counts_23_today.shape[0] - final_merged_df.shape[0]

In [ ]:
final_merged_df.head()

In [ ]:
# check sensor_name and Location
len(sorted(final_merged_df['New_Sensor_Name'].unique()))

In [ ]:
len(final_merged_df['Location'].unique())

In [ ]:
final_merged_df[final_merged_df['Location_ID'].isin([120, 111, 122, 124, 7, 123])][['New_Sensor_Name', 'Location']].drop_duplicates()

## <a id='toc7_3_'></a>[Split data based on covid period](#toc0_)

In [ ]:
split_and_save_data(final_merged_df, save_dir, save_subdirs.merged, rewrite=rewrite)

## <a id='toc7_4_'></a>[Split data based on year](#toc0_)

In [ ]:
for year, group in final_merged_df.groupby('Year'):
  save_data(group, save_dir, save_subdirs.merged, f"footfall_merged_for_year_{year}.xlsx", rewrite=rewrite)

# <a id='toc8_'></a>[Handling missing values (without interpolation)](#toc0_)
the data generated from this section is used for online learning

## <a id='toc8_1_'></a>[Extract year span for each sensor](#toc0_)

In [ ]:
save_path = save_dir / save_subdirs.missing_rate
save_path.mkdir(parents=True, exist_ok=True)
save_path

In [ ]:
df = pd.read_csv(save_dir /save_subdirs.merged / "footfall_merged.csv")

# Ensure Date_Time is a pandas datetime object
df['Date_Time'] = pd.to_datetime(df['Date_Time'])

# Group by sensor name and find time span for each sensor
# time_spans = df.groupby('Sensor_Name').agg({'Date_Time': ['min', 'max']})
# year_spans = df.groupby('Sensor_Name').agg({'Date_Time': [lambda x: x.dt.year.min(), lambda x: x.dt.year.max()]})
time_spans = df.groupby('New_Sensor_Name').agg({'Date_Time': ['min', 'max']})
year_spans = df.groupby('New_Sensor_Name').agg({'Date_Time': [lambda x: x.dt.year.min(), lambda x: x.dt.year.max()]})
year_spans.columns = ['Start_Year', 'End_Year']

unique_start_years = sorted(year_spans['Start_Year'].unique())

data_slices = {}

# For each unique starting year, grab data for the sensors active in that period
for i, start_year in enumerate(unique_start_years):
  # sensors starting before or during the current start year
  sensors_to_include = year_spans[year_spans['Start_Year'] <= start_year].index.tolist()

  if i == len(unique_start_years) - 1:
    end_year = df['Date_Time'].dt.year.max()  # Last year of the dataset
  else:
    end_year = unique_start_years[i + 1] - 1  # A year before the next starting year

  # Extract data
  # data_slice = df[(df['Sensor_Name'].isin(sensors_to_include)) &
  #                 (df['Date_Time'].dt.year >= start_year) &
  #                 (df['Date_Time'].dt.year <= end_year)]
  data_slice = df[(df['New_Sensor_Name'].isin(sensors_to_include)) &
                (df['Date_Time'].dt.year >= start_year) &
                (df['Date_Time'].dt.year <= end_year)]

  data_slices[(start_year, end_year)] = data_slice

In [ ]:
# Define a dictionary to store time span and corresponding sensors
time_span_and_sensors = {}

for (start_year, end_year), data_slice in data_slices.items():
  # sensors_in_slice = data_slice['Sensor_Name'].unique()
  sensors_in_slice = data_slice['New_Sensor_Name'].unique()
  time_span_and_sensors[(start_year, end_year)] = sensors_in_slice

print(f"The total time spans: {len(time_span_and_sensors)}")

for time_span, sensors in time_span_and_sensors.items():
  print(f"Time Span: {time_span[0]} - {time_span[1]} ({len(sensors)} sensors)")
  # print("Sensors:")
  # for sensor in sensors:
  #     print(f"  {sensor}")
  # print()

In [ ]:
# original data
data = df.pivot(index='New_Sensor_Name', columns='Date_Time', values='Hourly_Counts')
data.shape

In [ ]:
data.head()

In [ ]:
plot_time_series_data_iterative(df, save_path)

### <a id='toc8_1_1_'></a>[Save data](#toc0_)

In [ ]:
save_data(data, save_dir, save_subdirs.missing_rate, 'footfall_merged.csv', rewrite=rewrite)
save_data(time_spans, save_dir, save_subdirs.missing_rate, 'time_spans.xlsx', rewrite=rewrite)
save_data(year_spans, save_dir, save_subdirs.missing_rate, 'year_spans.xlsx', rewrite=rewrite)

# original data slices
for (start_year, end_year), slice_data in data_slices.items():
  save_data(slice_data, save_dir, save_subdirs.missing_rate, f"original_data_{start_year}_{end_year}.xlsx", rewrite=rewrite)

## <a id='toc8_2_'></a>[Calculate missing rate and plot data](#toc0_)

In [ ]:
missing_rates = {}

for (start_year, end_year), slice_data in data_slices.items():
  print(f"Processing slice {start_year} - {end_year}")
  grouped_slice_data = slice_data.pivot(index='New_Sensor_Name', columns='Date_Time', values='Hourly_Counts')
  save_data(grouped_slice_data, save_dir, save_subdirs.missing_rate, f"grouped_original_{start_year}_{end_year}.csv", rewrite=rewrite)

  # var_name = f"data_slice_{start_year}_{end_year}"
  # globals()[var_name] = slice_data

  missing_rate_per_sensor = grouped_slice_data.isnull().mean(axis=1).sort_values(ascending=False)
  print("Calculated missing rates")
  plot_missing_rate(missing_rate_per_sensor, start_year, end_year, save_path, rewrite=rewrite)
  print(f"Saved missing rate plot for slice {start_year} - {end_year}")

  plot_time_series_data(grouped_slice_data, start_year, end_year, save_path, rewrite=rewrite)
  print(f"Saved time series plot for slice {start_year} - {end_year}")
  plot_time_series_data_sensor(grouped_slice_data, start_year, end_year, save_path, rewrite=rewrite)
  plot_time_series_data_iterative(slice_data, save_path, start_year, end_year, True, rewrite=rewrite)
  print(f"Saved time series plot for each sensor for slice {start_year} - {end_year}")

## <a id='toc8_3_'></a>[Remove sensor with missing data rate >= 50%](#toc0_)

In [ ]:
group_processed_slices = []
processed_slices = []

for (start_year, end_year), slice_data in data_slices.items():
  print(f"Processing slice {start_year} - {end_year}")
  grouped_slice_data = slice_data.pivot(index='New_Sensor_Name', columns='Date_Time', values='Hourly_Counts')
  missing_rate_per_sensor = grouped_slice_data.isnull().mean(axis=1).sort_values(ascending=False)

  remove_sensors = missing_rate_per_sensor[missing_rate_per_sensor >= 0.5].index.values

  print(f"Data size before removing: {grouped_slice_data.shape}, {len(remove_sensors)} sensors will be removed.")
  grouped_slice_data.drop(remove_sensors, inplace=True)
  print(f"Data size after removing: {grouped_slice_data.shape}")

  save_data(grouped_slice_data, save_dir, save_subdirs.remove_sensors, f"grouped_original_{start_year}_{end_year}.csv", index=True, rewrite=rewrite)
  group_processed_slices.append(grouped_slice_data)

grouped_combined_data = pd.concat(group_processed_slices, axis=1)
print(f"Size of grouped_combined_data: {grouped_combined_data.shape}")

save_data(grouped_combined_data, save_dir, save_subdirs.remove_sensors, "grouped_combined_data.csv", index=True, rewrite=rewrite)

## <a id='toc8_4_'></a>[Create segments](#toc0_)
The sensors in each segment have same year span.

In [ ]:
df = pd.read_csv(save_dir / save_subdirs.remove_sensors / 'grouped_combined_data.csv')

df_melted = pd.melt(df, id_vars=['New_Sensor_Name'], var_name='Date_Time', value_name='Hourly_Counts')
df_melted['Date_Time'] = pd.to_datetime(df_melted['Date_Time'])
df_melted['Year'] = df_melted['Date_Time'].dt.year
df_melted = df_melted.dropna(subset=['Hourly_Counts'])

df = df_melted.copy()

# Create a mapping of sensor names to unique integer values
sensors = df['New_Sensor_Name'].unique()
sensor_mapping = {sensor: i for i, sensor in enumerate(sensors)}

# For each year, find which sensors are active
active_sensors_per_year = df.groupby('Year')['New_Sensor_Name'].unique()

# Create segments based on changes in active sensors set
segments = [(active_sensors_per_year.index[0], active_sensors_per_year.index[0])]
for year in active_sensors_per_year.index[1:]:
  # checks if the set of active sensors for the current year is the same as
  # the set of active sensors for the last year in the last segment
  if set(active_sensors_per_year[year]) == set(active_sensors_per_year[segments[-1][1]]):
    segments[-1] = (segments[-1][0], year) # upate the ending year
  else:
    segments.append((year, year))

print(len(segments))

In [ ]:
for segment in segments:
  subset = df[(df['Year'] >= segment[0]) & (df['Year'] <= segment[1])] # segment[0] <= df['Year'] <= segment[1]
  grouped_subset = subset.pivot(index='New_Sensor_Name', columns='Date_Time', values='Hourly_Counts')
  save_data(subset, save_dir, save_subdirs.final_group, f"data_{segment[0]}_{segment[1]}.xlsx", index=True, rewrite=rewrite)
  save_data(grouped_subset, save_dir, save_subdirs.final_group, f"grouped_data_{segment[0]}_{segment[1]}.csv", index=True, rewrite=rewrite)

# <a id='toc9_'></a>[Process features](#toc0_)

In [73]:
os.makedirs(save_dir / save_subdirs.add_fea, exist_ok=True)

sensor = pd.read_excel(save_dir / save_subdirs.merged / 'sensor_locations_processed.xlsx')

## <a id='toc9_1_'></a>[funs](#toc0_)

In [82]:
def haversine(lon1, lat1, lon2, lat2):
  """
  Calculate the great-circle distance between two points 
  on the Earth (specified in decimal degrees)
  """
  # Convert decimal degrees to radians
  lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

  # Haversine formula
  dlon = lon2 - lon1
  dlat = lat2 - lat1
  a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
  c = 2 * asin(sqrt(a))
  r = 6371  # Radius of Earth in kilometers. Use 3956 for miles
  return c * r

In [74]:
def fetch_amenities_count(lat, lon, distance, amenity_types):
  amenity_count = {}
  
  for amenity in amenity_types:
    # print(f"Fetching {amenity}...")
    try:
      gdf = ox.features_from_point((lat, lon), tags={'amenity': amenity}, dist=distance)
      if not gdf.empty:
        amenity_count[amenity] = len(gdf)
    except:
      pass
      
  return amenity_count

In [75]:
def sensor_amenities_count(sensor_df, distance, amenity_types):
  amenities_counts = []
  
  for index, row in sensor_df.iterrows():
    lat, lon = row['Latitude'], row['Longitude']
    count = fetch_amenities_count(lat, lon, distance, amenity_types)
    amenities_counts.append(count)
      
  amenities_count_df = pd.DataFrame(amenities_counts, index=sensor_df.index)
  return amenities_count_df

In [76]:
def fetch_amenities(lat, lon, distance, amenity_types):  
  gdfs = []
  for amenity in amenity_types:
    # print(f"Fetching {amenity}...")
    try:
      gdf = ox.features_from_point((lat, lon), tags={'amenity': amenity}, dist=distance)
      if not gdf.empty:
        gdf.reset_index(inplace=True)
        gdf = gdf[['osmid', 'amenity', 'geometry']]
        gdfs.append(gdf)
    except Exception as e:
      # print(f"Error fetching {amenity}: {e}")
      pass
  
  combined_gdf = pd.concat(gdfs, ignore_index=True) if gdfs else pd.DataFrame()

  return combined_gdf

## <a id='toc9_2_'></a>[fetch nearby amenity counts for each sensor](#toc0_)

In [ ]:
place_name = "Melbourne, Victoria, Australia"

# Fetch all geometries in the area
gdf = ox.geometries_from_place(place_name, tags={'amenity': True})

# Filter to keep only the 'amenity' tag
amenities = gdf[gdf['amenity'].notnull()]

amenity_types = amenities['amenity'].unique().tolist()

save_amenities = save_dir / save_subdirs.add_fea / 'amenity_types_melbourne.txt'
if not save_amenities.exists():
  with open(save_amenities, 'w') as f:
    for amenity in amenity_types:
      f.write(f"{amenity}\n")

  print("Amenity types saved to amenity_types_melbourne.txt")
else:
  amenity_types = pd.read_csv(save_amenities, header=None)[0].tolist()

we have 212 amenities in the city, and require 3 to 4 hrs to fetch all amenties for all sensors.

In [ ]:
# amenity_types = [
#   'arts_centre', 'atm', 'bank', 'bar', 'bbq', 'bench', 'bicycle_parking', 'bicycle_rental', 'bus_station',
#   'cafe', 'car_rental', 'car_wash', 'cinema', 'clinic', 'college', 'community_centre', 'dentist', 'doctor',
#   'drinking_water', 'fast_food', 'ferry_terminal', 'fire_station', 'fuel', 'grave_yard', 'gym', 
#   'hospital', 'ice_cream', 'kindergarten', 'library', 'marketplace', 'monastery', 'nightclub', 'nursing_home',
#   'parking', 'pharmacy', 'place_of_worship', 'police', 'post_box', 'post_office', 'pub', 'public_building',
#   'recycling', 'restaurant', 'school', 'shelter', 'shop', 'taxi', 'telephone', 'theatre', 'toilets', 
#   'townhall', 'university', 'vending_machine', 'veterinary', 'waste_basket', 'waste_disposal', 'train_station'
# ]

# amenity_types = ['restaurant', 'cafe']

distance = 1000  # 1000 meters radius

amenities_count_1000 = sensor_amenities_count(sensor, distance, amenity_types)
amenities_count_1000

In [ ]:
amenities_count_1000['Sensor_Name'] = sensor['New_Sensor_Name']
save_data(amenities_count_1000, save_dir, save_subdirs.add_fea, 'amenities_count_1000.csv')

In [ ]:
distance = 500

amenities_count_500 = sensor_amenities_count(sensor, distance, amenity_types)
amenities_count_500

In [ ]:
amenities_count_500['Sensor_Name'] = sensor['New_Sensor_Name']
save_data(amenities_count_500, save_dir, save_subdirs.add_fea, 'amenities_count_500.csv')

In [ ]:
distance = 100

amenities_count_100 = sensor_amenities_count(sensor, distance, amenity_types)
amenities_count_100

In [ ]:
amenities_count_100['Sensor_Name'] = sensor['New_Sensor_Name']
save_data(amenities_count_100, save_dir, save_subdirs.add_fea, 'amenities_count_100.csv')

## <a id='toc9_3_'></a>[fetch nearby amenities for each sensor](#toc0_)

Given:
- `S` = Set of all sensors.
- `D` = Set of distances `[100, 500, 1000]` meters.
- `A(lat, lon, d)` = Function that returns a set of amenities around the location (latitude `lat`, longitude `lon`) within distance `d`.

The total set of unique amenities fetched for each sensor at each distance can be expressed as:

$$
T = \bigcup_{d \in D} \bigcup_{s \in S} A(lat_s, lon_s, d)
$$

Where:
- `T` = Total set of unique amenities fetched.
- `lat_s, lon_s` = Latitude and longitude of sensor `s`.

This expression represents the union of all amenities sets fetched for each sensor at each of the distances. It's a union operation because duplicate amenities (those appearing in multiple sensors' vicinity or at different distances) are only counted once.

For counting the total number of unique amenities fetched, count the elements in `T` (i.e. summing up the amenities around each individual sensor.):

$$
\text{Total Unique Amenities} = |T|
$$


In [77]:
save_valid_amenities = save_dir / save_subdirs.add_fea / 'valid_amenity_types_melbourne.txt'
if not save_valid_amenities.exists():
  valid_amenities = pd.read_csv(save_dir / save_subdirs.add_fea / 'amenities_count_1000.csv')
  with open(save_valid_amenities, 'w') as f:
    for amenity in valid_amenities.columns:
      if amenity != 'Sensor_Name':
        f.write(f"{amenity}\n")
  
  valid_amenities = valid_amenities.columns
else:
  valid_amenities = pd.read_csv(save_valid_amenities, header=None)[0].tolist()
print(len(valid_amenities))

95


In [48]:
all_gdfs_ = []
sensor_gdfs = defaultdict(dict)

for distance in [100, 500, 1000]:
  print(distance)
  for index, row in sensor.iterrows():
    lat, lon = row['Latitude'], row['Longitude']
    gdfs = fetch_amenities(lat, lon, distance, valid_amenities)
    
    if not gdfs.empty:
      all_gdfs_.append(gdfs)
      sensor_gdfs[row['New_Sensor_Name']][distance] = gdfs['osmid'].tolist() # save all amenties
  
  valid_amenities_ = pd.concat(all_gdfs_, ignore_index=True) if all_gdfs else pd.DataFrame()
  valid_amenities_ = valid_amenities_.drop_duplicates()
  save_data(valid_amenities_, save_dir, save_subdirs.add_fea, f'valid_total_amenities_{distance}.csv')

100
500
1000


In [51]:
data_for_df = []
for sensor_name, distances in sensor_gdfs.items():
  for dist, osmids in distances.items():
    data_for_df.append({
        'Sensor_Name': sensor_name,
        'Distance': dist,
        'Osmids': osmids
    })

sensor_gdfs_df = pd.DataFrame(data_for_df)

save_data(sensor_gdfs_df, save_dir, save_subdirs.add_fea, f'valid_amenities_each_sensor.csv')

../data_preprocessed/5. join_features created.
../data_preprocessed/5. join_features/sensor_gdfs.csv will be updated.
../data_preprocessed/5. join_features/sensor_gdfs.csv saved.


Another way to fetch total amenties is finding total amenties in a circle (radius of the circle is set as the distance to this farthest sensor + extra_distance)

In [86]:
latitudes = sensor['Latitude'].values
longitudes = sensor['Longitude'].values
center_lat = np.mean(latitudes)
center_lon = np.mean(longitudes)

max_dist = max([haversine(lon, lat, center_lon, center_lat) for lat, lon in zip(latitudes, longitudes)])
print(max_dist)

for distance in [100, 500, 1000]:
  print(distance)
  extended_radius = max_dist * 1000 + distance 

  amenities_circle = fetch_amenities(center_lat, center_lon, extended_radius, valid_amenities)

  amenities_circle = amenities_circle.drop_duplicates()
  save_data(amenities_circle, save_dir, save_subdirs.add_fea, f'valid_menities_in_circle_{distance}.csv')


3.926927659977373
100
../data_preprocessed/5. join_features created.
../data_preprocessed/5. join_features/valid_menities_in_circle_100.csv will be updated.
../data_preprocessed/5. join_features/valid_menities_in_circle_100.csv saved.
500
../data_preprocessed/5. join_features created.
../data_preprocessed/5. join_features/valid_menities_in_circle_500.csv will be updated.
../data_preprocessed/5. join_features/valid_menities_in_circle_500.csv saved.
1000
../data_preprocessed/5. join_features created.
../data_preprocessed/5. join_features/valid_menities_in_circle_1000.csv will be updated.
../data_preprocessed/5. join_features/valid_menities_in_circle_1000.csv saved.


## Compare two methods

### 1. fetch total amenties in a circle

In [89]:
m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

# Plot each sensor on the map
for lat, lon in zip(latitudes, longitudes):
  folium.CircleMarker(
      location=[lat, lon],
      radius=5, # small radius for sensor points
      color='blue',
      fill=True,
      fill_color='blue'
  ).add_to(m)

# Plot the central point
folium.Marker(
  [center_lat, center_lon],
  popup='Center',
  icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

# Draw a circle with the extended radius
folium.Circle(
  [center_lat, center_lon],
  radius=max_dist * 1000 + 100, # radius in meters
  color='green',
  fill=True,
  fill_opacity=0.2
).add_to(m)

m

### 2. sum up nearby amenties for each sensor

In [107]:
sensor_circle_radius = 100

m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

for lat, lon in zip(latitudes, longitudes):
  folium.CircleMarker(
    location=[lat, lon],
    radius=5, 
    color='red',
    fill=True,
    fill_color='red'
  ).add_to(m)

  folium.Circle(
    location=[lat, lon],
    radius=sensor_circle_radius,
    color='blue',
    fill=False
  ).add_to(m)

m
